# Exploration des Résultats des Admis au BAC 2024 en République du Congo

Ce notebook présente une analyse exploratoire des résultats du BAC 2024 en République du Congo, réalisée en utilisant le langage SQL dans l'environnement Databricks. L'objectif principal de cette exploration est de comprendre les tendances des performances des étudiants à travers différentes séries (scientifique et littéraire) et de mettre en lumière les meilleures moyennes obtenues.

Nous avons utilisé plusieurs requêtes SQL pour extraire, et visualiser les données, permettant ainsi d'identifier les étudiants ayant obtenu les meilleures performances, ainsi que les moyennes les plus élevées par série. Ce travail offre une vue d'ensemble des résultats, facilitant une compréhension approfondie des performances académiques des admis au BAC 2024.

## LISTE DES FICHIERS (TABLES) DANS LA BASE

In [0]:
display(dbutils.fs.ls("dbfs:/FileStore/tables"))

path,name,size,modificationTime
dbfs:/FileStore/tables/BAC.csv,BAC.csv,4225622,1724508908000


In [0]:
# Spécifiez le chemin complet du fichier à supprimer
#file_path = "dbfs:/FileStore/tables/BAC.csv"

# Supprimez le fichier
#dbutils.fs.rm(file_path, True)


Out[2]: False

## SELECTION DE LA TABLE (FICHIER) A UTILISER

In [0]:
# import du fichier bac.csv

df=spark.read.csv("dbfs:/FileStore/tables/BAC.csv")

In [0]:
#Liste des variables dans le jeu de données
var = spark.sql ('''
show columns from bac
''')
var.show()

+-------------+
|     col_name|
+-------------+
|    MATRICULE|
|          NOM|
|       PRENOM|
|  DATE_DE_NCE|
|  LIEU_DE_NCE|
|         SEXE|
|         CODE|
|        SERIE|
|      MOYENNE|
|      MENTION|
|ETABLISSEMENT|
+-------------+



In [0]:

# Nombre total des admis toute serie confondue

TOTAL = spark.sql (''' 
   select count(matricule) AS NOMBRE_TOTAL_DES_ADMIS
    FROM BAC
''')
TOTAL.show()

+----------------------+
|NOMBRE_TOTAL_DES_ADMIS|
+----------------------+
|                 39876|
+----------------------+



## 10 PREMIERS TOUTE SERIE CONFONDUE



In [0]:

# Liste  des 10 premiers admis au BAC 2024

P10TSC = spark.sql('''
                   SELECT NOM, PRENOM, SEXE, MOYENNE, SERIE, ETABLISSEMENT
FROM BAC
ORDER BY MOYENNE DESC
LIMIT 10
                   ''')
P10TSC.show()

+------------------+--------------------+----+-------+-----+--------------------+
|               NOM|              PRENOM|SEXE|MOYENNE|SERIE|       ETABLISSEMENT|
+------------------+--------------------+----+-------+-----+--------------------+
|MAYANDZA MOUKANDZA|        Bruce Miguel|   M|   16.3|    C|LYCEE INTER-DEPAR...|
|            TEBELE|  Joëldie Belcharina|   F|  16.16|   A4|LYCEE INTER-DEPAR...|
|   KIHOULOU NDEMBO|      Manich Vichnel|   M|  16.08|    C|LYCEE INTER-DEPAR...|
|MBOUALA - NTSOUHOU|      Rebecca  Olive|   F|  16.03|    D|     C.S. EMMANUEL 1|
|         NGATSIEBE|      Richi Bernichi|   M|   16.0|    D|      LYCEE DE NGABE|
|           MAKONKI|Suzy Luchress Rap...|   M|  15.79|    D|LYCEE D'EXCELLENC...|
|            ONGOUI|       Maria Paulina|   F|  15.76|    D|CANDIDATS LIBRES ...|
|KOUANDZI-MOUKOUELO|        Frey Bechard|   M|  15.76|    C|LYCEE INTER-DEPAR...|
|       MOUEBAKOLET|      Pachico Estime|   M|   15.7|    D|     LYCEE DE OUESSO|
|           ONDO

## ADMIS AVEC LA DEUXIEME MOYENNE LA PLUS ELEVEE

In [0]:
# Admis avec la deuxième moyenne la plus elevée

SHM = spark.sql('''
SELECT BAC.MATRICULE, NOM, PRENOM, SEXE, SERIE, MOYENNE AS SECOND_HIGHEST_AVG, BAC.ETABLISSEMENT
FROM BAC
WHERE MOYENNE = (
    SELECT MAX(MOYENNE)
    FROM BAC
    WHERE MOYENNE < (SELECT MAX(MOYENNE) FROM BAC)
);
               ''')

SHM.show()

+-----------+------+------------------+----+-----+------------------+--------------------+
|  MATRICULE|   NOM|            PRENOM|SEXE|SERIE|SECOND_HIGHEST_AVG|       ETABLISSEMENT|
+-----------+------+------------------+----+-----+------------------+--------------------+
|A4ABQ240010|TEBELE|Joëldie Belcharina|   F|   A4|             16.16|LYCEE INTER-DEPAR...|
+-----------+------+------------------+----+-----+------------------+--------------------+




## LISTE DES ADMIS AVEC LA MOYENNE LA PLUS ELEVEE PAR SERIE



In [0]:
# Liste des meuilleurs pour chaque serie, premier de chaque serie uniquement

MOYSERIE = spark.sql( ''' 
SELECT NOM, BAC.PRENOM, BAC.SERIE, MOYENNE
FROM BAC
INNER JOIN (
    SELECT SERIE, MAX(MOYENNE) AS MAX_MOYENNE
    FROM BAC
    GROUP BY SERIE
) AS SUBQUERY
ON BAC.SERIE = SUBQUERY.SERIE AND BAC.MOYENNE = SUBQUERY.MAX_MOYENNE
ORDER BY BAC.MOYENNE DESC 
''')

MOYSERIE.show()

+------------------+------------------+-----+-------+
|               NOM|            PRENOM|SERIE|MOYENNE|
+------------------+------------------+-----+-------+
|MAYANDZA MOUKANDZA|      Bruce Miguel|    C|   16.3|
|            TEBELE|Joëldie Belcharina|   A4|  16.16|
|MBOUALA - NTSOUHOU|    Rebecca  Olive|    D|  16.03|
|         MANTOUARI|Romelique Anderson|   A2|  14.62|
|            ILENDA|  Divine Celestine|   A3|  11.86|
+------------------+------------------+-----+-------+



## TP10 SERIE C



In [0]:
# les 10 meuilleurs de la serie C

TOP10C =spark.sql(''' 
                  SELECT NOM, PRENOM, SEXE, MOYENNE, SERIE, ETABLISSEMENT
FROM BAC
WHERE SERIE ="C"
ORDER BY MOYENNE DESC
LIMIT 10
                  ''')
TOP10C.show()

+------------------+--------------------+----+-------+-----+--------------------+
|               NOM|              PRENOM|SEXE|MOYENNE|SERIE|       ETABLISSEMENT|
+------------------+--------------------+----+-------+-----+--------------------+
|MAYANDZA MOUKANDZA|        Bruce Miguel|   M|   16.3|    C|LYCEE INTER-DEPAR...|
|   KIHOULOU NDEMBO|      Manich Vichnel|   M|  16.08|    C|LYCEE INTER-DEPAR...|
|KOUANDZI-MOUKOUELO|        Frey Bechard|   M|  15.76|    C|LYCEE INTER-DEPAR...|
|  MAMPAKA MABOULOU|    Dreech Merveille|   M|  15.66|    C|LYCEE INTER-DEPAR...|
|          MBOUNGOU|    Cherubin Hulrich|   M|   15.6|    C|LYCEE INTER-DEPAR...|
|              YIKA|Emmanuel Boderich...|   M|  15.49|    C|LYCEE INTER-DEPAR...|
|        GOMA MBANI|       Claude Arsène|   M|  15.36|    C|LYCEE INTER-DEPAR...|
|             EKOKI|    Franspéra Junior|   M|  15.35|    C|LYCEE INTER-DEPAR...|
|     IBAïBE OKOMBI|     Brayane Shékina|   M|  15.35|    C|     LYCEE CHAMINADE|
|         MONGWA

##TOP10 SERIE D



In [0]:
# Liste des 10 meuilleurs de la serie D
TOP10D =spark.sql(''' 
                  SELECT NOM, PRENOM, SEXE, MOYENNE, SERIE, ETABLISSEMENT
FROM BAC
WHERE SERIE ="D"
ORDER BY MOYENNE DESC
LIMIT 10
                  ''')

TOP10D.show()

+------------------+--------------------+----+-------+-----+--------------------+
|               NOM|              PRENOM|SEXE|MOYENNE|SERIE|       ETABLISSEMENT|
+------------------+--------------------+----+-------+-----+--------------------+
|MBOUALA - NTSOUHOU|      Rebecca  Olive|   F|  16.03|    D|     C.S. EMMANUEL 1|
|         NGATSIEBE|      Richi Bernichi|   M|   16.0|    D|      LYCEE DE NGABE|
|           MAKONKI|Suzy Luchress Rap...|   M|  15.79|    D|LYCEE D'EXCELLENC...|
|            ONGOUI|       Maria Paulina|   F|  15.76|    D|CANDIDATS LIBRES ...|
|       MOUEBAKOLET|      Pachico Estime|   M|   15.7|    D|     LYCEE DE OUESSO|
|           ONDONGO|         Chris Vialy|   M|  15.68|    D|CAND. LIBRES DE B...|
|             NGUIE|       Bonheur Divin|   M|  15.64|    D|      GS VICTOR HUGO|
|    ITOUA MOULANGA|   Landricia Aminata|   F|  15.52|    D|LYCEE CATHOLIQUE ...|
|      TSATY MAKAYA|   Verole Jubertrine|   F|  15.46|    D|LYCEE D'EXCELLENC...|
|            TAN

## TOP10 SERIE A4

Liste des 10 meuilleurs de la serie A4

In [0]:
# Liste des 10 meuilleurs de la serie A4
TOP10A4=spark.sql(''' 
                  SELECT NOM, PRENOM, SEXE, MOYENNE, SERIE, ETABLISSEMENT
FROM BAC
WHERE SERIE ="A4"
ORDER BY MOYENNE DESC
LIMIT 10
                  ''')

TOP10A4.show()

+------------------+--------------------+----+-------+-----+--------------------+
|               NOM|              PRENOM|SEXE|MOYENNE|SERIE|       ETABLISSEMENT|
+------------------+--------------------+----+-------+-----+--------------------+
|            TEBELE|  Joëldie Belcharina|   F|  16.16|   A4|LYCEE INTER-DEPAR...|
|         MAVOUNGOU|     Carla Nephtalie|   F|  15.66|   A4|         GSP L ALPHA|
|   KATALI MILANDOU|   Lucie Anne Sophie|   F|  15.58|   A4|    GS SACRE - COEUR|
|   EDOUARD MFOUROU|     Nellie Rogeltty|   F|   15.5|   A4|     LYCEE CHAMINADE|
|             MBIZI|Will  Jewel Mac- ...|   M|  15.44|   A4|CS BILINGUE LIGHT...|
|TCHINGOUTA KAYEMBE|        Brèche Sarah|   F|  15.42|   A4|   DOM HELDER CAMARA|
| NDAKOUZI LUSADISU|Gloire Jeanne Bon...|   F|  15.36|   A4|LYCEE INTER-DEPAR...|
|             KONGO|        Gräce Divine|   F|  15.36|   A4|       GS EMMANCEL 2|
|             GANGA|        Dan Andrersy|   M|   15.3|   A4|     LYCEE CHAMINADE|
|          LONDZ

## TOP10 SERIE A2

In [0]:
# Liste des 10 meuilleurs de la serie A2

TOP10A2=spark.sql('''
                  SELECT NOM, PRENOM, SEXE, MOYENNE, SERIE, ETABLISSEMENT
FROM BAC
WHERE SERIE ="A2"
ORDER BY MOYENNE DESC
LIMIT 10
                  ''')

TOP10A2.show()

+-------------------+--------------------+----+-------+-----+--------------------+
|                NOM|              PRENOM|SEXE|MOYENNE|SERIE|       ETABLISSEMENT|
+-------------------+--------------------+----+-------+-----+--------------------+
|          MANTOUARI|  Romelique Anderson|   M|  14.62|   A2|CAND. LIBRES DE B...|
|        MAYEKE YOKA|Constant Mignon Noël|   M|   14.5|   A2|SEMINAIRE SAINT P...|
|     ZONIABA  ABBAH|   Henrica Clémencia|   F|  14.38|   A2| ANNE MARIE JAVOUHEY|
|              TOURE|              Mamadi|   M|  14.34|   A2|G. S. DOM HELDER ...|
|   ZOGUIWALO POYINA|  Dominique Emmanuel|   M|  13.94|   A2|CS BIENHEUREUSE M...|
| KIBANGARY NTONDELE|     Mondesir Guelvy|   M|  13.92|   A2| SEMINAIRE DE LOANGO|
|           HABIMANA|            Alfreide|   M|  13.78|   A2|SEMINAIRE SAINT P...|
|TCHOKONGUEU SITCHEU|Christ Mignon Nou...|   M|   13.6|   A2|G. S. DOM HELDER ...|
|       SAMBA VIANEY|        Isaac Aloïse|   M|  13.56|   A2|SEMINAIRE SAINT P...|
|   

## TOP10 SERIE A3

In [0]:
# Liste des meuilleurs de la serie A3, 

TOP10A3=spark.sql('''
                  SELECT NOM, PRENOM, SEXE, MOYENNE, SERIE, ETABLISSEMENT
FROM BAC
WHERE SERIE ="A3"
ORDER BY MOYENNE DESC
LIMIT 10
                  ''')

TOP10A3.show()

+------+----------------+----+-------+-----+------------------+
|   NOM|          PRENOM|SEXE|MOYENNE|SERIE|     ETABLISSEMENT|
+------+----------------+----+-------+-----+------------------+
|ILENDA|Divine Celestine|   F|  11.86|   A3|CS LEAURRY-FLORIDE|
+------+----------------+----+-------+-----+------------------+



## EFFECTIF PAR MAOYENNE

In [0]:
# effectif total pour chaque moyenne allant de 10 à 16

df2=spark.sql('''
              
              SELECT FLOOR(MOYENNE) AS MOYENNE_TRONQUEE, COUNT(*) AS EFFECTIF
              FROM BAC
              GROUP BY MOYENNE_TRONQUEE
              ''')
df2.show()

+----------------+--------+
|MOYENNE_TRONQUEE|EFFECTIF|
+----------------+--------+
|              10|   14529|
|              12|    8794|
|              11|   13911|
|              13|    2060|
|              14|     505|
|              15|      72|
|              16|       5|
+----------------+--------+



## NOMBRE DES ADMIS PAR SERIE

In [0]:
# Effectif total des admis par serie

df3=spark.sql('''
              
              SELECT SERIE, COUNT(*) AS NB_ADMIS_PAR_SERIE
              FROM BAC
              GROUP BY SERIE
              ORDER BY NB_ADMIS_PAR_SERIE DESC
              ''')
df3.show()

+-----+------------------+
|SERIE|NB_ADMIS_PAR_SERIE|
+-----+------------------+
|    D|             23948|
|   A4|             13983|
|    C|              1867|
|   A2|                77|
|   A3|                 1|
+-----+------------------+



## EFFECTIF DES ADMIS PAR ETABLISSEMENT

In [0]:

# effectif total des admis par ordre alphabetique
TOPETS = spark.sql('''          
         SELECT ETABLISSEMENT, count(ETABLISSEMENT) AS TOTAL_ETS
FROM bac
GROUP BY ETABLISSEMENT
ORDER BY TOTAL_ETS DESC
LIMIT 15
''')
TOPETS.show()

+--------------------+---------+
|       ETABLISSEMENT|TOTAL_ETS|
+--------------------+---------+
|CAND. LIBRES DE B...|     2376|
|LYCEE THOMAS SANK...|     1083|
|LYCEE THOMAS SANK...|      797|
|LYCEE DE LA REVOL...|      730|
|CAND. LIBRES DE P...|      725|
|LYCEE SEBASTIEN M...|      644|
|    LYCEE DE MPAKA I|      643|
|  LYCEE 30 MARS 1970|      642|
|    LYCEE DE KINTELE|      626|
|LYCEE NGANGA EDOUARD|      609|
|LYCEE DE LA RECON...|      584|
|LYCEE J. B. TATY ...|      577|
|     LYCEE CHAMINADE|      540|
|   LYCEE DE MADINGOU|      528|
|      LYCEE DE NKAYI|      525|
+--------------------+---------+

